In [2]:
import pandas as pd
import mysql.connector

In [3]:
# Common function defenition
def connect_db():
    db_connection = mysql.connector.connect(
    host='146.56.43.43',
    user='ffbemaster',
    password='aksaksgo1!',
    database='ffbe')
    return db_connection
def fetch_data_from_db(sql):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    cursor.close()
    conn.close()
    return res
def fetch_data_from_db_with_col_names(sql):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    col_names = cursor.column_names
    cursor.close()
    conn.close()
    return res, col_names
def write_to_server(sql, values):
    conn = connect_db()
    cursor = conn.cursor()
    if type(values) != list:
        cursor.execute(sql, values)
    else:
        for i in range(len(values)):
            cursor.execute(sql, values[i])
    res = cursor.fetchall()
    cursor.close()
    conn.commit()
    conn.close()
    return res

In [12]:
sql_query = """SELECT * FROM char_list"""
data, col_names = fetch_data_from_db_with_col_names(sql_query)
chars = pd.DataFrame(data, columns=col_names)
chars.columns

Index(['seq', 'char_name', 'char_jp_name', 'char_trsl_name', 'char_g8_link',
       'char_img_src', 'char_eval', 'char_rarity', 'char_element',
       'char_element_img_src', 'char_main_job', 'char_main_job_class',
       'char_main_job_class_alias', 'char_sub_job1', 'char_sub_job2'],
      dtype='object')

In [13]:
def get_char_name(character):
    if character['char_name']:
        return character['char_name']
    if character['char_trsl_name']:
        return character['char_trsl_name']
    if character['char_jp_name']:
        return character['char_jp_name']
    else:
        return "NoName"

In [6]:
print(get_char_name(chars.iloc[2]))

박쥐


In [ ]:
selected_chars = pd.DataFrame()
for i, c in chars.iterrows():
    print(get_char_name(c))
    selected_chars = pd.concat([selected_chars, c], ignore_index=True)
    if i == 4:
        chars.drop(i)
    if i > 10:
        break
selected_chars
print(chars)

In [21]:
def show_chars_in_brief(chars:pd.DataFrame, selected_chars, width=100):
    # chars = Dataframe
    if type(selected_chars) != list:
        selected_chars = [selected_chars]
    col_num = 4
    cnt = 0
    element_list = ['화', '빙', '풍', '토', '뇌', '수', '명', '암']
    chars_on_element = {}
    for e in element_list:
        #속성별로 나누어 담는다.
        chars_on_element[e] = chars[(chars['char_element']==e) 
                                    & (chars['char_main_job_class_alias'].isin(selected_chars))]
    for e in element_list:
        if len(chars_on_element[e]):
            print(f"for {e}", "-"*50)
            print(chars_on_element[e]['char_trsl_name'])

In [24]:
show_chars_in_brief(chars, "글로브")

for 화 --------------------------------------------------
26       케프카
72    로이 머스탱
Name: char_trsl_name, dtype: object
for 뇌 --------------------------------------------------
40    헬레나(수영복)
Name: char_trsl_name, dtype: object
